In [1]:
import os
import pandas as pd
from datetime import datetime
import re
import openpyxl
from openpyxl.styles import PatternFill

In [2]:
# Define Directories
working_dir = r'C:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Inventory\WH Inventory Reports\Working_files'
output_dir = r'C:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Inventory\WH Inventory Reports\Reports'

# Get current date
current_date = datetime.now().strftime("%Y-%m-%d")

# Find the current month's files 
files = [f for f in os.listdir(working_dir) if f.endswith(".xlsx")]

# Read and combine data
all_data = []
for file in files:
    file_path = os.path.join(working_dir,file)

    # Extract date from filename
    match = re.search(r'SAP_vs_Warehouse_all_data_(\d{8})_\d{4}', file)
    if match:
        date_str = match.group(1) # e.g. "20250328"
        report_date = datetime.strptime(date_str, '%Y%m%d')
        try:
            date_label = report_date.strftime('%#m/%#d_QTY')
        except:
            date_label = report_date.strftime('%m/%d_QTY') # fallback with leading zeros 
    else:
        # fallback: use modified date
        mod_time = datetime.fromtimestamp(os.path.getmtime(file_path))
        date_label = mod_time.strftime('%#m/%#d_QTY')

    try:
        df = pd.read_excel(file_path, dtype={"Batch": str})
        df['Source File'] = file
        df['Date_Label'] = date_label
        df['Report Date'] = report_date
        all_data.append(df[['Plant', 'Storage Location', 'Material', 'Description', 'Batch', 'Qty_SAP', 'Qty_WH', 'Delta', 'Date_Label', 'Report Date',
                            'Manufacture Date', 'Critical Shelf Life Date', 'Expiration date', 'Number of Summers', 'Unit Value', 'Pallet number']])
    except Exception as e:
        print(f"Failed to process {file}: {e}")

In [3]:
# Combine into single DataFrame
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)

    # Get the last known Storage Location for each batch
    combined_df_sorted = combined_df.sort_values(by='Report Date', ascending=False)
    last_snapshot = combined_df_sorted.drop_duplicates(subset=['Plant', 'Material', 'Batch'])[['Plant','Material','Batch','Storage Location']].copy()
    last_snapshot.rename(columns={'Storage Location': 'Last Storage Location'}, inplace=True)

    # Pivot into wide format
    qty_wide = combined_df.pivot_table(
        index=['Plant', 'Material', 'Batch', 'Description'],
        columns='Date_Label',
        values='Qty_WH',
        aggfunc='first',
        fill_value=0
    ).reset_index

    # Get expiration and shelf life from first occurance
    extra_info = combined_df.groupby(['Plant', 'Material', 'Batch', 'Description'], as_index=False).agg({
        'Expiration date': 'first',
        'Critical Shelf Life Date': 'first'
    })

    print(type(qty_wide))

    # Merge everything
    merged_df = qty_wide.merge(extra_info, on=['Plant', 'Material', 'Batch', 'Description'], how='left')
    merged_df = merged_df.merge(last_snapshot, on=['Plant', 'Material', 'Batch'], how='left') 

    # Sort columns for clean ordering
    qty_columns = sorted([col for col in merged_df.columns if '_QTY' in col],
                         key=lambda x:datetime.strptime(x.replace('_QTY',''), '%m/%d'))
    final_columns = ['Plant', 'Material', 'Batch', 'Description', 'Expiration date', 'Critical Shelf Life Date', 'Last Storage Location'] + qty_columns
    merged_df = merged_df[final_columns]

    # Add movement column: 'Static' if all QTYs are the same, else 'Changed'
    def detect_movement(row):
        values = row[qty_columns].tolist()
        return 'Static' if all (v == values[0] for v in values) else 'Changed'
    
    merged_df['Batch Movement'] = merged_df.apply(detect_movement, axis=1)
    final_columns.append['Batch Movement']
    merged_df = merged_df[final_columns]

    # Sort rows for readability
    merged_df.sort_values(by=['Plant', 'Material', 'Batch'], inplace=True)

    # Split into tabs by plant
    plant_list = ['BP01', 'BP02', 'BP04', 'BP07']
    output_path = os.path.join(output_dir, f"Batch_Movement_Report_{current_date}.xlsx")


    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        for plant in plant_list:
            plant_df = merged_df[merged_df['Plant'] == plant].copy()
            if plant_df.empty:
                pd.DataFrame(columns=final_columns).to_excel(writer, sheet_name=plant, index=False)
            else:
                plant_df.to_excel(writer, sheet_name=plant, index=False)
            
    # Apply alternating row colors by material
    fill_gray = PatternFill(start_color='F2F2F2', end_color="F2F2F2", fill_type='solid')

    wb = openpyxl.load_workbook(output_path)
    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        ws.freeze_panes = 'A2'
        ws.auto_filter.ref = ws.dimensions

        previous_material = None
        use_gray = False

        for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
            current_material = row[1].value # Material is in column B
            if current_material != previous_material:
                use_gray = not use_gray
                previous_material = current_material

            if use_gray:
                for cell in row:
                    cell.fill = fill_gray        

    wb.save(output_path)
    wb.close()
    print(f"Batch Movement Report generated: {output_path}")
else:
    print("No data to process")
   

<class 'method'>


AttributeError: 'function' object has no attribute 'merge'